In [1]:
import pandas as pd

df_enrolment   = pd.read_csv("final_enrolment_clean.csv")
df_demographic = pd.read_csv("df_demographic_all_final.csv")
df_biometric   = pd.read_csv("final_biometric_cleaned.csv")


In [2]:
df_enrolment.shape, df_demographic.shape, df_biometric.shape

((1006007, 7), (1861102, 6), (1861107, 6))

In [3]:
print(df_enrolment.columns)
print(df_demographic.columns)
print(df_biometric.columns)

Index(['date', 'state', 'district', 'pincode', 'enro_age_0_5', 'enro_age_5_17',
       'enro_age_18_greater'],
      dtype='object')
Index(['date', 'pincode', 'demo_age_5_17', 'demo_age_17_', 'state',
       'district'],
      dtype='object')
Index(['date', 'pincode', 'bio_age_5_17', 'bio_age_17_', 'state', 'district'], dtype='object')


In [4]:
for df in [df_enrolment, df_demographic, df_biometric]:
    df["date"] = pd.to_datetime(df["date"], errors="coerce").dt.date
    df["pincode"] = df["pincode"].astype(str).str.strip()

In [5]:
df_enrolment_g = df_enrolment.groupby(["date","pincode"], as_index=False).sum(numeric_only=True)
df_demographic_g = df_demographic.groupby(["date","pincode"], as_index=False).sum(numeric_only=True)
df_biometric_g = df_biometric.groupby(["date","pincode"], as_index=False).sum(numeric_only=True)


In [6]:
merged_df = df_demographic_g.merge(df_biometric_g, on=["date","pincode"], how="outer")
merged_df = merged_df.merge(df_enrolment_g, on=["date","pincode"], how="outer")


In [7]:
pincode_map = (
    df_enrolment[["pincode","state","district"]]
    .dropna()
    .drop_duplicates(subset=["pincode"])
)

merged_df = merged_df.merge(pincode_map, on="pincode", how="left")


In [8]:
count_cols = [
    "demo_age_5_17","demo_age_17_",
    "bio_age_5_17","bio_age_17_",
    "enro_age_0_5","enro_age_5_17","enro_age_18_greater"
]

merged_df[count_cols] = merged_df[count_cols].fillna(0).round(0).astype("int64")
merged_df[["state","district"]] = merged_df[["state","district"]].fillna("Unknown")

print("shape:", merged_df.shape)
merged_df.head()


shape: (723885, 11)


,date,pincode,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,enro_age_0_5,enro_age_5_17,enro_age_18_greater,state,district
0,2025-01-03,110001,0,0,191,247,0,0,0,delhi,new delhi
1,2025-01-03,110002,65,714,260,427,0,0,0,delhi,central delhi
2,2025-01-03,110003,75,571,347,411,0,0,0,delhi,central delhi
3,2025-01-03,110005,0,0,441,950,0,0,0,delhi,central delhi
4,2025-01-03,110006,0,0,901,1526,0,0,0,delhi,north delhi


In [9]:
merged_df["date"] = pd.to_datetime(merged_df["date"], errors="coerce")

In [10]:
merged_df["year"] = merged_df["date"].dt.year
merged_df["day"] = merged_df["date"].dt.day
merged_df["month"] = merged_df["date"].dt.strftime("%B")
merged_df["weekday"] = merged_df["date"].dt.strftime("%A")

In [11]:
merged_df = merged_df.drop(columns=["date"])

In [12]:
order = [
    "pincode","state","district","day", "month", "weekday","year",
    "demo_age_5_17", "demo_age_17_",
    "bio_age_5_17", "bio_age_17_",
    "enro_age_0_5", "enro_age_5_17", "enro_age_18_greater"
]

merged_df = merged_df[order]
merged_df.head()

,pincode,state,district,day,month,weekday,year,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_,enro_age_0_5,enro_age_5_17,enro_age_18_greater
0,110001,delhi,new delhi,3,January,Friday,2025,0,0,191,247,0,0,0
1,110002,delhi,central delhi,3,January,Friday,2025,65,714,260,427,0,0,0
2,110003,delhi,central delhi,3,January,Friday,2025,75,571,347,411,0,0,0
3,110005,delhi,central delhi,3,January,Friday,2025,0,0,441,950,0,0,0
4,110006,delhi,north delhi,3,January,Friday,2025,0,0,901,1526,0,0,0


In [13]:
merged_df["weekday"].unique()

array(['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Thursday',
       'Wednesday'], dtype=object)

In [14]:
merged_df["state"].unique()

array(['delhi', 'haryana', 'uttar pradesh', 'punjab', 'chandigarh',
       'himachal pradesh', 'Unknown', 'jammu and kashmir', 'ladakh',
       'uttarakhand', 'rajasthan', 'gujarat',
       'dadra and nagar haveli and daman and diu', 'maharashtra', 'goa',
       'madhya pradesh', 'chhattisgarh', 'telangana', 'andhra pradesh',
       'puducherry', 'karnataka', 'tamil nadu', 'kerala', 'lakshadweep',
       'west bengal', 'sikkim', 'andaman and nicobar islands', 'odisha',
       'assam', 'meghalaya', 'arunachal pradesh', 'manipur', 'mizoram',
       'nagaland', 'tripura', 'bihar', 'jharkhand'], dtype=object)

In [16]:
merged_df = merged_df[merged_df["state"] != "Unknown"]

In [17]:
merged_df.to_csv(r"C:\Users\arsha\OneDrive\Desktop\UIDAY Hacakathon\merged_df.csv", index=False)
print("Saved")

Saved
